# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Предварительная оценка

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('./datasets/data.csv')
data.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
data.children.value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [5]:
data.gender.value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

In [6]:
data[data['days_employed'] >= 21900].shape

(3445, 12)

In [7]:
data[data['days_employed'] <= 0].shape

(15906, 12)

In [8]:
data[data['dob_years'] == 0].shape

(101, 12)

In [9]:
data.education.value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [10]:
data.duplicated().sum()

54

**Вывод**

  - В столбце _children_ есть записи, как однозначно ошибочные ('-1' с количеством записей **47**), так и имеющие все признаки таковых ('20' с количеством записей **76**)
  - В столбце _days_employed_:
    - **15906** записей с отрицательным значением, что представляет собой более половины значений;
    - **2174** записи со значением NaN (равно как и в _total_income_);
    - **3445** записей имеют значение больше 21900 дней (60 лет по 365 дней в году), что является заведомо ошибочным;
    - записи имеют тип данных float, что является не совсем практичным с точки зрения характера хранимых там данных.
  - В столбце _dob_years_ присутствует **101** запись со значением '0', что является заведемой ошибкой в рамках конкретной задачи ("младенец, получающий кредит").
  - В столбце _education_ записи идут с разным регистром, что усложняет оценку. Столбец _education_id_ никаких противоречий не вызывает, и является дублирующим для стобца _education_, имеющим числовое значение вместо строкового.
  - Столбцы _family_status_ и _family_status_id_ - в точной аналогии со стобцами _education_
  - Стобец _gender_ имеет одну запись нестандартную запись вида 'XNA'
  - Стобцы _income_type_  и _debt_ не вызывают никаких противоречий
  - количество строк дубликатов составляет **54**

## Шаг 2. Предобработка данных

### Обработка пропусков

#### Приведение к общему виду

Перед заполнением пропусков обработаем единичные ошибки и приведём таблицу к большему единообразию, в том числе:

- Приведение строк к нижнему регистру

In [11]:
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()

- Замена значения XNA в стобце gender на наиболее часто встречающийся

In [12]:
data.loc[data['gender'] == 'XNA', 'gender'] = 'F'

- Замена отрицательного количества детей на 0

In [13]:
data.loc[data['children'] < 0, 'children'] = 0

#### Заполнение пропусков

Заполнение пропусков (**2174** записи) и слишком больших значений (**3445**) в _days_employed_ произведём по принципу предположительного максимального рабочего стажа (начиная с 16 лет, 365 дней в году).

In [14]:
data['days_employed'].fillna((data['dob_years']-16)*365, inplace=True)
data.loc[data['days_employed'] >= 21900, 'days_employed'] = (data['dob_years']-16)*365

Записи с отрицательным значением, ввиду их большого количества (***15906*** записей из **21525** всего, что составляет значительные 74% всех записей), преобразуем путём отбрасывания "минуса", т.е. берём модуль значения.

In [15]:
data.loc[data['days_employed'] < 0, 'days_employed'] = abs(data['days_employed'])

In [16]:
##Код ревьюера
data_pension = data.copy()
data_pension.groupby('income_type')['total_income'].median()

income_type
безработный        131339.751676
в декрете           53829.130729
госслужащий        150447.935283
компаньон          172357.950966
пенсионер          118514.486412
предприниматель    499163.144947
сотрудник          142594.396847
студент             98201.625314
Name: total_income, dtype: float64

Заполнение пропусков в _total_income_ выполняем с использования медианного значением по группам.

In [17]:
data['total_income'] = data.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.median()))

Проверка результата

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Пропуски в столбце _days_employed_ и "странные" значения были заполнены с учётом предположений, основанных на формальной логике. Отрицательные же значения можно предположить, как верные, но с ошибкой на уровне формирования базы, добавившей "-" к записям.
Пропуски в стобце _total_income_ были заменены на медианные значения.

Возможные причины появления данных пропусков:
- пустые значения - ошибка ввода данных (человеческий фактор), или потеря данных (технологическая ошибка)
- "странные значения" - также двояко: технологическая ошибка в случае ошибки программы, используемой для заполнения (передавала неверные результаты в результате ошибок программирования), человеческий фактор - при обработке подобных данных в программах типа Excel при переводе типа ячейки с дат на числовые значения, к примеру.

### Замена типа данных

Повторная проверка типов данных столбцов _days_employed_ и _total_income_

In [19]:
print('Тип данных колонки days_employed:', data['days_employed'].dtype)
print('Тип данных колонки total_income:', data['total_income'].dtype)

Тип данных колонки days_employed: float64
Тип данных колонки total_income: float64


Отброс малозначимых чисел после запятой с преобразование в целое.

In [20]:
data['days_employed'] = data['days_employed'].dropna().astype('int64')
data['total_income'] = data['total_income'].dropna().astype('int64')

Оценка результата

In [21]:
print('Тип данных колонки days_employed:', data['days_employed'].dtype)
print('Тип данных колонки total_income:', data['total_income'].dtype)

Тип данных колонки days_employed: int64
Тип данных колонки total_income: int64


In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Тип данных float64 в колонках _days_employed_ и _total_income_ был заменён на int64 методом ***astype()*** библиотеки Pandas, так как в суть данных, содержащихся в этих колонках представляют из себя дни и денежные единицы, которые можно округлить до целого без потерь для результатов анализа.

### Обработка дубликатов

Оценим количество полных дубликатов с использованием функционала библиотеки Pandas

In [23]:
data.duplicated().sum()

71

Количество дубликоватов по сравнению с первоначальной оценкой выросло с 54 до 71, что является явным результатом наших предыдущих действий. Удалим их методом Pandas drop_duplicates() со сбросом индексов.

In [24]:
data = data.drop_duplicates().reset_index(drop = True)

Проверка:

In [25]:
data.duplicated().sum()

0

**Вывод**

Полные дубликаты в количестве 71 штуки были удалены из таблицы. Причинами появления дубликатов кака правило является  ошибка, связанная с написанием

### Лемматизация

Загрузка библиотеки pymystem3 для лемматизации значений колонки _purpose_.

In [26]:
from pymystem3 import Mystem
from collections import Counter

m = Mystem()

Необязательный шаг, получение списка ключевых слов для написания функции

In [27]:
test_1 = set(data.purpose.unique().tolist())
key_words = set()
for word in test_1:
    key_words.update(m.lemmatize(word))
print(key_words)

{'свадьба', 'покупка', 'приобретение', 'профильный', 'проведение', 'жилье', 'образование', 'с', 'со', 'получение', 'свой', 'жилой', 'семья', 'операция', 'подержанный', 'для', 'сдача', 'строительство', 'коммерческий', 'сделка', ' ', 'подержать', 'собственный', 'на', 'заниматься', 'недвижимость', '\n', 'дополнительный', 'ремонт', 'сыграть', 'высокий', 'автомобиль'}


Функция ***purpose_checker*** для дальнейшей обработки таблицы

In [28]:
def purpose_checker(row):
    purpose_to_check = m.lemmatize(row['purpose'])
    if 'свадьба' in purpose_to_check or 'сыграть' in purpose_to_check:
        return 'wedding'
    elif 'коммерческий' in purpose_to_check or 'сдача' in purpose_to_check: #новые строки взамен старых
        return 'commercial'
    elif 'жилье' in purpose_to_check or 'жилой' in purpose_to_check or 'недвижимость' in purpose_to_check: #новые строки взамен старых
        return 'home'
    elif 'автомобиль' in purpose_to_check:
        return 'car'
    elif 'образование' in purpose_to_check:
        return 'study'
    else:
        return None  

Применяем полученную функцию для обработки значений колонки *purpose*...

In [29]:
data['purpose'] = data.apply(purpose_checker, axis=1)

... и проверяем результат

In [30]:
data.purpose.value_counts()

home          8849
car           4306
study         4013
wedding       2324
commercial    1962
Name: purpose, dtype: int64

Проверяем, не появилось ли новых дубликатов...

In [31]:
data.duplicated().sum()

277

... и убираем их.

In [32]:
data = data.drop_duplicates().reset_index(drop = True)

**Вывод**

С использованием лемматизации, мы свели разные формулировки одной и той же задачи (цели кредита), к всего лишь пяти основным:
- жильё
- коммерческая недвижимость
- движимое имущество (автомобиль)
- образование
- свадьба
Также в результате подобной обраобтки мы получили ещё 277 дубликатов, которые так же были удалены.

### Категоризация данных

Последняя не категоризованная часть таблицы - это возраст, а точнее деление на возрастные группы, в связи с чем, переизпользуем код из занятий, и добавим его с небольшими доработками (разный возраст выхода на пенсию)

In [33]:
def age_group(row):
    age = row['dob_years']
    sex = row['gender']
    if age < 17:
        return 'children'
    elif age < 22:
        return 'student'
    elif (age < 60 and sex == 'F') or (age < 65 and sex == 'M'):
        return 'adult'
    else:
        return 'elder'

Применяем функцию построчно, а результат добавляем в качестве новой колонки, смотрим на результат

In [34]:
data['age_group'] = data.apply(age_group, axis=1)

In [35]:
data.age_group.value_counts()

adult       18810
elder        2091
student       176
children      100
Name: age_group, dtype: int64

**Вывод**

В результате этой категоризации мы смогли выделить группы людей по возрастам, которые необходимы для оценки кредитоспособности в будущем, и как основной результат, мы выяснили, что большая часть списка - взрослые люди, являющиеся основной надёжной клиентурой банка.

## Шаг 3. Ответы на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Создаю урезанную копию основной таблицы с добавлением столбца `child_not_free` со значениями *True* и *False*, обозначающие есть ли дети у заёмщика.

In [36]:
data_shrink = data[['debt', 'children', 'family_status', 'purpose', 'total_income']].copy()
data_shrink['child_not_free'] = data_shrink['children'].apply(lambda x : True if (x > 0) else False)
data_shrink.head()

,debt,children,family_status,purpose,total_income,child_not_free
0,0,1,женат / замужем,home,253875,True
1,0,1,женат / замужем,car,112080,True
2,0,0,женат / замужем,home,145885,False
3,0,3,женат / замужем,study,267628,True
4,0,0,гражданский брак,wedding,158616,False


Создаю сводную таблицу на основе урезанной, с количеством записей по каждому из вариантов наличия детей и долга и расчётным процентом не вернувших этот долг

In [37]:
data_pivot_child = data_shrink.pivot_table(index='child_not_free', columns='debt', aggfunc='size')

data_pivot_child['percentage_of_debt'] = ((data_pivot_child[1] / (data_pivot_child[0] + data_pivot_child[1])))

data_pivot_child.style.format({'percentage_of_debt': "{:.1%}"})

debt,0,1,percentage_of_debt
child_not_free,,,
False,12852,1062,7.6%
True,6586,677,9.3%


**Вывод**

Процент должников с детьми, чуть выше, чем должников без детей, но в общем, в обоих случаях укладывается до 10% всех случаев.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Создаю сводную таблицу на основе урезанной из прошлого шага, с количеством записей по каждому из вариантов семейного положения, наличия долга и расчётным процентом не вернувших этот долг

In [38]:
data_pivot_family = data_shrink.pivot_table(index='family_status', columns='debt', aggfunc='size')

data_pivot_family['percentage_of_debt'] = ((data_pivot_family[1] / (data_pivot_family[0] + data_pivot_family[1])))

data_pivot_family.style.format({'percentage_of_debt': "{:.1%}"})

debt,0,1,percentage_of_debt
family_status,,,
в разводе,1108,85,7.1%
вдовец / вдова,883,63,6.7%
гражданский брак,3739,388,9.4%
женат / замужем,11191,929,7.7%
не женат / не замужем,2517,274,9.8%


**Вывод**

Наибольшее количество задолжников в соотвествии с их семейным положением находится в категориях "гражданский брак" и "не в браке", но даже в этом случае, общее количество задолжников не выбивается за 10% порог от общих невозвращённых вовремя кредитов

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Уровень дохода мы будем оценивать относительно 33-го и 66-го процентиля общих значений общего дохода `total_income`.

In [39]:
quantile_low = data_shrink['total_income'].quantile([.33]).iloc[0]
quantile_high = data_shrink['total_income'].quantile([.66]).iloc[0]
print(quantile_low)
print(quantile_high)

118514.0
172357.0


Добавляем функцию для разделения на группы по значению дохода

In [40]:
def income_group(row):
    money = row['total_income']
    if money >= quantile_high:
        return 'rich'
    elif money <= quantile_low:
        return 'poor'
    else:
        return 'middle'

Применяем её

In [41]:
data_shrink['how_rich'] = data_shrink.apply(income_group, axis=1)

Создаю сводную таблицу на основе урезанной с добавленной колонкой `how_rich`

In [42]:
data_pivot_income = data_shrink.pivot_table(index='how_rich', columns='debt', aggfunc='size')

data_pivot_income['percentage_of_debt'] = ((data_pivot_income[1] / (data_pivot_income[0] + data_pivot_income[1])))

data_pivot_income.head().style.format({'percentage_of_debt': "{:.1%}"})

debt,0,1,percentage_of_debt
how_rich,,,
middle,6087,602,9.0%
poor,6430,579,8.3%
rich,6921,558,7.5%


**Вывод**

Процент задолжников падает с ростом дохода, но общее количество задолжников также не выбивается за 10% порог от общих невозвращённых вовремя кредитов 

- Как разные цели кредита влияют на его возврат в срок?

Создаю сводную таблицу на основе урезанной из прошлого шага, с количеством записей по каждому из вариантов целей кредита, наличия долга, и расчётным процентом не вернувших этот долг

In [43]:
data_pivot_purpose = data_shrink.pivot_table(index='purpose', columns='debt', aggfunc='size')

data_pivot_purpose['percentage_of_debt'] = ((data_pivot_purpose[1] / (data_pivot_purpose[0] + data_pivot_purpose[1])))

data_pivot_purpose.style.format({'percentage_of_debt': "{:.1%}"})

debt,0,1,percentage_of_debt
purpose,,,
car,3869,402,9.4%
commercial,1808,151,7.7%
home,8047,630,7.3%
study,3594,370,9.3%
wedding,2120,186,8.1%


**Вывод**

Наибольшее количество должников среди желающих купить новый автомобиль, и заняться обучением, наименьший же среди тех, кто решил купить дом, но опять же, менее 10% процентов от общего количества.

## Шаг 4. Общий вывод

Как легко заметить, общее количество должников всегда достаточно стабильно, и составляет не более 10% от категории, что совпадает с общим количеством значений должников, составляющих 8.9% от всех учтённых записей.
Семейное положение клиента, количество детей, его заработок и цели кредита влияют на факт погашения кредита в срок, но на значение, не превышающее среднее.